# Experiment 1

### Environment Setup

In [1]:
%%bash -s

# set root folder and get environment
export root="/home/fabric/work"
source $root/env

In [6]:
# imports
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed_extensions.fablib.node import Node
from threading import Thread
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import uuid
from collections import defaultdict

# constants
root = "/home/fabric/work"

# component names
slice_name = "experiment1"
server_site, client_site = fablib.get_random_sites(count=2)

# switch detail
client_switch_name = "client-switch"
server_switch_name = "client-switch"
switch_cores = 2
switch_ram = 8
switch_disk = 100

# host details
client_host_name = "client-host"
server_host_name = "server-host"
image = 'default_ubuntu_20'
server_host_subnet=IPv4Network('10.0.1.0/24')
server_host_addr=IPv4Address('10.0.1.1')
client_host_subnet=IPv4Network('10.0.2.0/24')
client_host_addr=IPv4Address('10.0.2.2')

# network details
net_server_switch_name = "net-server-switch"
net_client_switch_name = "net-client-switch"
net_switch_switch_name = "net-switch-switch"

In [3]:
# some helper functions
__id_counter = defaultdict(int)
def id(name=None):
  global __id_counter
  __id_counter[name] += 1
  return f"{name}-{str(__id_counter[name]).zfill(4)}"

## Setup Network

In [4]:

# create the slice
slice = fablib.new_slice(name=slice_name)

# add server switch
server_switch = slice.add_node(name=server_switch_name, site=server_site, image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
server_switch_iface_host = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
server_switch_iface_client_switch = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client switch
client_switch = slice.add_node(name=client_switch_name, site=client_site, image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
client_switch_iface_host = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
client_switch_iface_server_switch = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add server host
server_host = slice.add_node(name=server_host_name, site=server_site, image=image)
server_host_iface = server_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client host
client_host = slice.add_node(name=client_host_name, site=client_site, image=image)
client_host_iface = client_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add networks
server_switch_net = slice.add_l2network(name=net_server_switch_name, interfaces=[server_switch_iface_host, server_host_iface])
client_switch_net = slice.add_l2network(name=net_client_switch_name, interfaces=[client_switch_iface_host, client_host_iface])
switch_switch_net = slice.add_l2network(name=net_switch_switch_name, interfaces=[server_switch_iface_client_switch, client_switch_iface_server_switch])

# submit slice for creation
slice.submit()



-----------  ------------------------------------
Slice Name   experiment1
Slice ID     5fb6746c-8711-4350-a010-582c3247ef9a
Slice State  StableOK
Lease End    2022-10-31 17:56:24 +0000
-----------  ------------------------------------

Retry: 27, Time: 324 sec

ID                                    Name           Site    Host                          Cores    RAM    Disk  Image              Management IP                        State    Error
------------------------------------  -------------  ------  --------------------------  -------  -----  ------  -----------------  -----------------------------------  -------  -------
26968fc9-4ab8-420f-b696-c8bd727a6b11  server-switch  UTAH    utah-w4.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fe02:98eb  Active
df6c954c-aefd-4b4c-b0ad-3d6fa427057b  client-switch  FIU     fiu-w4.fabric-testbed.net         2      8     100  default_ubuntu_20  131.94.57.51                         Active
dbe85644-be4f-

In [7]:
# config threads
config_threads: dict[Node,Thread] = {}

In [10]:
# config information
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/"

# configure server host
server_host = slice.get_node(name=server_host_name)        
if type(ip_address(server_host.get_management_ip())) is IPv6Address:
    server_host.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
server_host_os_iface = server_host.get_interface(network_name=net_server_switch_name)
server_host_os_iface.ip_addr_add(addr=server_host_addr, subnet=server_host_subnet)
config_threads[server_host] = server_host.execute_thread(host_config_script)

# configure client host
client_host = slice.get_node(name=client_host_name)        
if type(ip_address(client_host.get_management_ip())) is IPv6Address:
    client_host.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
client_host_os_iface = client_host.get_interface(network_name=net_client_switch_name)
client_host_os_iface.ip_addr_add(addr=client_host_addr, subnet=client_host_subnet)
config_threads[client_host] = client_host.execute_thread(host_config_script)

In [11]:
# print the output of all nodes
for node, thread in config_threads.items():
  stdout, stderr = thread.result()
  print(f"Config thread node {node.get_name()} complete")
  print(f"stdout:\n{stdout}")
  print(f"stderr:\n{stderr}")

Config thread node server-host complete
stdout:
Selecting previously unselected package python3-backcall.
(Reading database ... 63577 files and directories currently installed.)
Preparing to unpack .../00-python3-backcall_0.1.0-2_all.deb ...
Unpacking python3-backcall (0.1.0-2) ...
Selecting previously unselected package python3-decorator.
Preparing to unpack .../01-python3-decorator_4.4.2-0ubuntu1_all.deb ...
Unpacking python3-decorator (4.4.2-0ubuntu1) ...
Selecting previously unselected package python3-parso.
Preparing to unpack .../02-python3-parso_0.5.2-1ubuntu1_all.deb ...
Unpacking python3-parso (0.5.2-1ubuntu1) ...
Selecting previously unselected package python3-jedi.
Preparing to unpack .../03-python3-jedi_0.15.2-1_all.deb ...
Unpacking python3-jedi (0.15.2-1) ...
Selecting previously unselected package python3-pickleshare.
Preparing to unpack .../04-python3-pickleshare_0.7.5-2_all.deb ...
Unpacking python3-pickleshare (0.7.5-2) ...
Selecting previously unselected package pyth

## Delete Slice

In [ ]:
raise
slice = fablib.get_slice(name=slice_name)
slice.delete()